# Speeding the Code Up to 300 times Faster Compared with Original Code

In [2]:
import time
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

## 1. Original Code
- All the descriptions are removed for the clear comparision
- The source of the functions are ASM.py file
- Due to the difference in unit (ft/sec) and axis setting (x-position and time stamp starts from 0, proceeds to positive value), the hyper parameters are adjusted

- Original paper that ASM.py is based on: An adaptive smoothing method for traffic state identification from incomplete information [link to paper](https://link.springer.com/chapter/10.1007/978-3-662-07969-0_33)

- smooth_x_window $\sigma$
    - Paper → 0.60 km = 0.373 mi = 1,968.5 ft
    - Example code → 0.24 km = 0.150 mi = 792.0 ft
    - 1,000 ft is used
- smooth_t_window $\tau$
    - Paper → 1.1 min = 66 sec
    - Example code → 0.6 min = 36 sec
    - 60 sec is used
- c_free
    - Paper → +80 km/h = +49.7 mi/h = +72.9 ft/s
    - Example code → -69.2 km/h = -43.0 mi/h = -63.1 ft/s
    - Based on the data analysis, 90 ft/s is used
- c_cong
    - Paper → -15.0 km/h = -9.3 mile/h = -13.7 ft/s
    - Example code → +20.9 km/h = +13 mile/h = +19.1 ft/s
    - Based on the data analysis, -15 ft/s is used
- V_c
    - Paper → 60 km/h → 37.3 mile/h → 54.7 ft/s
    - Example code → 60 km/h → approximated to 36.0 mile/h → 52.8 ft/s
    - 55 ft/s  is used
- $\Delta V$ (Transition region)
    - Paper → 20 km/h → 12.43 mile/h → 18.2269 ft/s
    - Example code → 20 km/h → 12.43 mile/h → 18.2269 ft/s
    - 18.227 ft/s  is used

In [3]:
def beta_free_flow(x, t, x_s, t_s, x_win, t_win, c_free=90):

    dt = t-t_s- (x-x_s)/c_free
    dx = x-x_s
    return np.exp(-(np.abs(dx)/x_win + np.abs(dt)/t_win))


def beta_cong_flow(x, t, x_s, t_s, x_win, t_win, c_cong=-15):

    dt = t-t_s- (x-x_s)/c_cong
    dx = x-x_s
    return np.exp(-(np.abs(dx)/x_win + np.abs(dt)/t_win))


def EGTF(x, t, smooth_x_window, smooth_t_window, speed_raw_df):

    speed = speed_raw_df[(np.abs(speed_raw_df.t - t)<=(smooth_t_window/2)) & (np.abs(speed_raw_df.x - x)<=(smooth_x_window/2))]
        
    speed = speed.copy()
    EGTF_v_free = 80
    EGTF_v_cong = 80

    speed['beta_free'] = speed.apply(
        lambda v: beta_free_flow(x, t, v.x, v.t, smooth_x_window, smooth_t_window),
        axis=1)
    speed['beta_cong'] = speed.apply(
        lambda v: beta_cong_flow(x, t, v.x, v.t, smooth_x_window, smooth_t_window),
        axis=1)
    if((sum(speed.beta_free)!=0) & (sum(speed.beta_cong)!=0)):
        EGTF_v_free = sum(speed.beta_free * speed.speed) / sum(speed.beta_free)
        EGTF_v_cong = sum(speed.beta_cong * speed.speed) / sum(speed.beta_cong)
    v = min(EGTF_v_free,EGTF_v_cong)
    tanh_term = np.tanh((55-v) / 18.227)
    w = 0.5*(1+tanh_term)
    return w*EGTF_v_cong + (1-w)*EGTF_v_free


def smooth_raw_data(speed_raw, dx, dt, smooth_x_window=1000, smooth_t_window=60):

    EGTF_speed = pd.DataFrame(speed_raw)
    EGTF_speed.columns=['t_index','x_index','speed']
    EGTF_speed['t'] = dt*EGTF_speed['t_index']
    EGTF_speed['x'] = dx*EGTF_speed['x_index']
    speed_raw_df  = EGTF_speed.dropna().copy()
    tqdm.pandas(desc="Processing EGTF")
    EGTF_speed['EGTF'] = EGTF_speed.progress_apply(lambda v: EGTF(v.x, v.t, smooth_x_window,smooth_t_window,speed_raw_df), axis=1)
    EGTF_speed.columns = ['t','x','raw_speed','time','milemarker','speed']
    return EGTF_speed

- The speed_mat file is a 2D array. 
- X-axis is a time-axis and Y-axis is a x-position.
- The value of each element is speed.
- For a quick and clear comparision, use the part of the original array (100 by 120)
- 100 by 120 is equal to 100 sec by 1200 ft

In [3]:
'''
select your own path to the speed matrix, then adjust the code
'''

# speed_mat = np.load('westbound_11_29_21_23_lane_1.npy')

# cut 100 by 120
# speed_mat = speed_mat[:100, :120]

# time_w, space_w = speed_mat.shape

# t_col = np.arange(time_w).repeat(space_w)
# x_col = np.tile(np.arange(space_w), time_w)

# speed_df = pd.DataFrame({'t': t_col, 'x': x_col, 'speed': speed_mat.flatten()})
# speed_df.to_csv('selected/lane_matrix/westbound_11_29_21_23_speed_raw_lane_1_slice.csv', index=False)

In [4]:
speed_df = pd.read_csv('westbound_11_29_21_23_speed_raw_lane_1_slice.csv')

dx = 10 
dt = 1

t0 = time.time()
smooth_speed = smooth_raw_data(speed_df, dx, dt, 1000, 60)
print(f"Smoothing compute time: {time.time() - t0}s")

smooth_speed.to_csv('westbound_11_29_21_23_speed_raw_lane_1_smooth_1.csv', index=False)

Processing EGTF:   0%|          | 0/12000 [00:00<?, ?it/s]

Smoothing compute time: 1173.5075566768646s


- Total time consumption can vary, but usually around 1,000 sec

## 2. Updated Code

In [10]:
speed_df = pd.read_csv('westbound_11_29_21_23_speed_raw_lane_1_slice.csv')
speed_raw = speed_df.speed.values.reshape(100, 120)

t0 = time.time()

time_w, space_w = speed_raw.shape

x_scale = 10
t_win = 60 # 30 sec
x_win = 100 # 1000 ft
c_free = 90
c_cong = -15

# create time matrix and space matrix
time_mat = np.arange(time_w).repeat(space_w).reshape(time_w, -1)
space_mat = np.asarray([np.arange(space_w)]*(time_w)) * x_scale

speed_smooth = np.zeros((time_w, space_w))

for time_idx in range(time_w):
    for space_idx in range(space_w):

        EGTF_v_free = 80
        EGTF_v_cong = 80

        # time_idx and real time is 1:1, space_idx and real space is 1:10
        time_val = time_idx
        space_val = space_idx * x_scale

        t_start = max(time_idx-(t_win//2), 0)
        t_end = min(time_idx+(t_win//2)+1, time_w)
        
        x_start = max(space_idx-(x_win//2), 0)
        x_end = min(space_idx+(x_win//2)+1, space_w)
        
        speed_slice = speed_raw[t_start:t_end, x_start:x_end]
        time_slice = time_mat[t_start:t_end, x_start:x_end]
        space_slice = space_mat[t_start:t_end, x_start:x_end]
        
        time_diff = time_val-time_slice
        dx_mat = space_val-space_slice

        dt_mat_free = time_diff - dx_mat/c_free
        dt_mat_cong = time_diff - dx_mat/c_cong

        beta_mat_free = np.exp(-(np.abs(dt_mat_free)/t_win + np.abs(dx_mat)/(x_win*x_scale)))
        beta_mat_cong = np.exp(-(np.abs(dt_mat_cong)/t_win + np.abs(dx_mat)/(x_win*x_scale)))
        
        if (sum(sum(beta_mat_free)) != 0) and (sum(sum(beta_mat_cong)) != 0):
            EGTF_v_free = sum(sum(beta_mat_free * speed_slice)) / sum(sum(beta_mat_free))
            EGTF_v_cong = sum(sum(beta_mat_cong * speed_slice)) / sum(sum(beta_mat_cong))
        
        v = min(EGTF_v_free, EGTF_v_cong)
        tanh_term = np.tanh((55 - v) / 18.227)
        w = 0.5*(1+tanh_term)
        speed_smooth[time_idx, space_idx] = w*EGTF_v_cong + (1-w)*EGTF_v_free

t_col = np.arange(0, 100).repeat(120)
x_col = np.tile(np.arange(0, 120), 100)

print(f"Smoothing compute time: {time.time() - t0}s")

speed_df = pd.DataFrame({'t': t_col, 'x': x_col, 'raw': speed_raw.flatten(), 'speed': speed_smooth.flatten()})
speed_df.to_csv('westbound_11_29_21_23_speed_raw_lane_1_smooth_2.csv', index=False)

Smoothing compute time: 3.3159537315368652s


- Total time consumption is about 3.33 sec.
- Using matrix-wise computation and indexing, the speed of the code got 300 times faster.
- Output file shows the exact same results.